# Insta360 Calibration

In [12]:
import cv2 as cv
import os
import numpy as np

In [11]:
cap = cv.VideoCapture("vids/calib3.mp4")
if not cap.isOpened():
    print("Cannot open vid")
    exit()

frames = []
while True:
    ret, last_frame = cap.read()
    if not ret:
        break
    frames.append(last_frame)

h, w = frames[0].shape[:2]

for i, frame in enumerate(frames):
    frames[i] = frames[i][:, w//2:]
    
h, w = frames[0].shape[:2]

cv.imwrite("calibration_image.jpg", frames[0])

cap.release()

In [13]:
pattern_width = 9
pattern_height = 6
pattern_size = (pattern_width, pattern_height)

world_points = np.zeros((np.prod(pattern_size), 3), np.float32)
world_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)

world_points

array([[0., 0., 0.],
       [1., 0., 0.],
       [2., 0., 0.],
       [3., 0., 0.],
       [4., 0., 0.],
       [5., 0., 0.],
       [6., 0., 0.],
       [7., 0., 0.],
       [8., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [2., 1., 0.],
       [3., 1., 0.],
       [4., 1., 0.],
       [5., 1., 0.],
       [6., 1., 0.],
       [7., 1., 0.],
       [8., 1., 0.],
       [0., 2., 0.],
       [1., 2., 0.],
       [2., 2., 0.],
       [3., 2., 0.],
       [4., 2., 0.],
       [5., 2., 0.],
       [6., 2., 0.],
       [7., 2., 0.],
       [8., 2., 0.],
       [0., 3., 0.],
       [1., 3., 0.],
       [2., 3., 0.],
       [3., 3., 0.],
       [4., 3., 0.],
       [5., 3., 0.],
       [6., 3., 0.],
       [7., 3., 0.],
       [8., 3., 0.],
       [0., 4., 0.],
       [1., 4., 0.],
       [2., 4., 0.],
       [3., 4., 0.],
       [4., 4., 0.],
       [5., 4., 0.],
       [6., 4., 0.],
       [7., 4., 0.],
       [8., 4., 0.],
       [0., 5., 0.],
       [1., 5., 0.],
       [2., 5

In [14]:
tracked_frames = []
all_corners = []
all_world_points = []

for i, frame in enumerate(frames):
    print(f"Tracking frame {i}...", flush=True)
    grey_scale = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    found_corners, corners = cv.findChessboardCorners(grey_scale, (pattern_width, pattern_height))
    if found_corners:
        tracked = cv.drawChessboardCorners(grey_scale, (pattern_width, pattern_height), corners, ret)
        tracked_frames.append(tracked)
        all_corners.append(corners)
        all_world_points.append(world_points)

print(f"Tracked {len(tracked_frames)} frames.")
cv.imwrite("tracked.jpg", tracked_frames[0])

Tracking frame 0...
Tracking frame 1...
Tracking frame 2...
Tracking frame 3...
Tracking frame 4...
Tracking frame 5...
Tracking frame 6...
Tracking frame 7...
Tracking frame 8...
Tracking frame 9...
Tracking frame 10...
Tracking frame 11...
Tracking frame 12...
Tracking frame 13...
Tracking frame 14...
Tracking frame 15...
Tracking frame 16...
Tracking frame 17...
Tracking frame 18...
Tracking frame 19...
Tracking frame 20...
Tracking frame 21...
Tracking frame 22...
Tracking frame 23...
Tracking frame 24...
Tracking frame 25...
Tracking frame 26...
Tracking frame 27...
Tracking frame 28...
Tracking frame 29...
Tracking frame 30...
Tracking frame 31...
Tracking frame 32...
Tracking frame 33...
Tracking frame 34...
Tracking frame 35...
Tracking frame 36...
Tracking frame 37...
Tracking frame 38...
Tracking frame 39...
Tracking frame 40...
Tracking frame 41...
Tracking frame 42...
Tracking frame 43...
Tracking frame 44...
Tracking frame 45...
Tracking frame 46...
Tracking frame 47...
Tr

True

In [ ]:
np_world_points = [np.array(p).reshape(-1, 1, 3) for p in all_world_points]
np_corners = [np.array(c).reshape(-1, 1, 2) for c in all_corners]

flags = cv.fisheye.CALIB_FIX_SKEW + cv.fisheye.CALIB_USE_INTRINSIC_GUESS + cv.fisheye.CALIB_FIX_K3 + cv.fisheye.CALIB_FIX_K4 +cv.fisheye.CALIB_FIX_PRINCIPAL_POINT 
criteria = (cv.TERM_CRITERIA_EPS+cv.TERM_CRITERIA_MAX_ITER, 500, 1e-6)

K0 = np.array([[500, 0, w/2],
               [0, 500, h/2],
               [0, 0, 1]], dtype=np.float64)
D0 = np.zeros((4, 1), dtype=np.float64)

_, camera_matrix, dist_coefs, _, _ = cv.fisheye.calibrate(np_world_points, np_corners, (w,h), K0, D0, flags=flags, criteria=criteria)
print(camera_matrix)
print(dist_coefs)

[[437.10995055   0.         540.        ]
 [  0.         463.87061871 540.        ]
 [  0.           0.           1.        ]]
[[ 0.53147818]
 [-0.24110855]
 [ 0.        ]
 [ 0.        ]]


In [18]:
Kn = cv.fisheye.estimateNewCameraMatrixForUndistortRectify(
    K=camera_matrix,
    D=dist_coefs,
    image_size=(w, h),
    R=np.eye(3),
    balance=0.0,
    new_size=(w, h)
)

img_undistorted = cv.fisheye.undistortImage(
    frames[0],
    K=camera_matrix,
    D=dist_coefs,
    Knew=Kn
)

cv.imwrite("undistorted.jpg", img_undistorted)

True

In [19]:
# undistort test image from video
cap = cv.VideoCapture("vids/down_frontback.mp4")
if not cap.isOpened():
    print("Cannot open vid")
    exit()

ret, frame = cap.read()

cap.release()

frame_h, frame_w = frame.shape[:2]

frame = frame[:, :frame_w//2]
    
img_undistorted = cv.fisheye.undistortImage(
    frame,
    K=camera_matrix,
    D=dist_coefs,
    Knew=Kn
)

cv.imwrite("distorted1.jpg", frame)
cv.imwrite("undistorted1.jpg", img_undistorted)



True

# EgoWholeBody Calibration

In [9]:
import cv2
import numpy as np

In [10]:
img = cv2.imread('/home/max/Documents/RA/datasets/mo2cap2/img/fc2_save_2017-10-11-135418-0158.jpg')
h, w = img.shape[:2]

K = np.array([[500, 0.0, w/2],
                   [0.0, 500, h/2],
                   [0.0, 0.0, 1.0]])

D=np.array([-0.125, 0.01, 0.0, 0.0])

Kn = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(
    K=K,
    D=D,
    image_size=(w, h),
    R=np.eye(3),
    balance=1.0,
    new_size=(w, h)
)

img_undistorted = cv2.fisheye.undistortImage(
    img,
    K=K,
    D=D,
    Knew=Kn
)

cv2.imwrite("undistorted.jpg", img_undistorted)

True